In [17]:
import pandas as pd
import numpy as np
import scipy
from statsmodels import stats as stats_inbuilt

from scipy import stats

In [18]:
data = pd.read_csv('lesson_1.csv', sep=',')
data

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,STT5750_Breast_021_DCIS,cancer,1.466240,3.689955,21.331981,19.359546,23.505609,11.576176,11.325851,80.572057,...,2.807635,1.466240,2.807635,1.466240,2.807635,3.689955,18.449777,1.466240,1.466240,1.466240
68,STT5751_Breast_021_IDC,cancer,2.492410,12.279444,30.087759,17.703926,26.064890,10.707081,11.520120,65.041865,...,2.492410,2.492410,1.301619,3.275668,1.301619,1.301619,8.601194,1.301619,1.301619,1.301619
69,STT5758_Breast_022_IDC,cancer,3.530477,20.006038,53.924651,25.449565,30.949995,16.794696,17.121366,70.958462,...,1.176826,1.176826,1.176826,2.961613,1.176826,1.176826,10.764365,1.176826,1.176826,1.176826
70,STT5763_Breast_022_IDC,cancer,3.733734,8.860505,32.538666,21.585069,24.987992,11.460224,11.201202,67.767125,...,2.840946,1.483635,1.483635,1.483635,1.483635,1.483635,8.522837,1.483635,1.483635,1.483635


In [19]:
# Task 1

In [20]:
data_normal = data[data['Diagnosis'] == 'normal']
data_early = data[data['Diagnosis'] == 'early neoplasia']
data_cancer = data[data['Diagnosis'] == 'cancer']

In [21]:
normal_early_active = []
early_cancer_active = []

for prop in data.columns.drop(['Diagnosis', 'Patient_id']):
    prop_1 = data_normal[prop]
    prop_2 = data_early[prop]
    result = scipy.stats.ttest_ind(prop_1, prop_2, equal_var = False)
    
    normal_early_active.append(result[1])
        
        
for prop in data.columns.drop(['Diagnosis', 'Patient_id']):
    prop_1 = data_early[prop]
    prop_2 = data_cancer[prop]
    result = scipy.stats.ttest_ind(prop_1, prop_2, equal_var = False)
    
    early_cancer_active.append(result[1])

In [22]:
# Answer for task 1
print(1575)
print(3490)

1575
3490


In [23]:
# Task 2

In [24]:
import statsmodels.stats.multitest as smm

In [25]:
first_hyp_arr = smm.multipletests(normal_early_active, alpha=0.025, method='holm')[0]
second_hyp_arr = smm.multipletests(early_cancer_active, alpha=0.025, method='holm')[0]

In [26]:
def fold_change(C, T):
    mean_1 = C.mean()
    mean_2 = T.mean()
    
    if mean_2 > mean_1:
        return mean_2 / mean_1
    elif mean_2 < mean_1:
        return -(mean_1 / mean_2)
    

first_significantly_changed = 0
for index in np.where(first_hyp_arr == True)[0]:
    values_1 = data_normal.iloc[:, index + 2]
    values_2 = data_early.iloc[:, index + 2]
    
    if abs(fold_change(values_1, values_2)) > 1.5:
        first_significantly_changed += 1
        
second_significantly_changed = 0
for index in np.where(second_hyp_arr == True)[0]:
    values_1 = data_early.iloc[:, index + 2]
    values_2 = data_cancer.iloc[:, index + 2]
    
    if abs(fold_change(values_1, values_2)) > 1.5:
        second_significantly_changed += 1

In [27]:
print(first_significantly_changed, second_significantly_changed)

2 77


In [28]:
# Task 3

In [29]:
first_hyp_arr = smm.multipletests(normal_early_active, alpha=0.025, method='fdr_bh')[0]
second_hyp_arr = smm.multipletests(early_cancer_active, alpha=0.025, method='fdr_bh')[0]

In [30]:
first_significantly_changed = 0
for index in np.where(first_hyp_arr == True)[0]:
    values_1 = data_normal.iloc[:, index + 2]
    values_2 = data_early.iloc[:, index + 2]
    
    if abs(fold_change(values_1, values_2)) > 1.5:
        first_significantly_changed += 1
        
second_significantly_changed = 0
for index in np.where(second_hyp_arr == True)[0]:
    values_1 = data_early.iloc[:, index + 2]
    values_2 = data_cancer.iloc[:, index + 2]
    
    if abs(fold_change(values_1, values_2)) > 1.5:
        second_significantly_changed += 1

In [31]:
print(first_significantly_changed, second_significantly_changed)

4 524
